In [1]:
import torch
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')


Using cuda for inference


## MODEL TRAINING

In [2]:
!pip install torchviz
!pip install torchsummary
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=6be52432e9c3c733caaea431b4a717fdecc4f0bea83ba3531d9a3a4c247b3730
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=58a8a9cb2bf0e2a0974db201ed7a7932e3426cec165fa8cbfa107c5ada3c1c02
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [3]:
import torchvision.datasets as datasets
normalize= transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732], std=[0.24697122, 0.24338895, 0.2615926 ])
transform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
        # transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732],std=[0.24697122, 0.24338895, 0.2615926 ])
    ])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:04<00:00, 35066592.78it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b4', pretrained=False)
model.eval().to(device)

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=48, out_features=12, bias=True)
          (expand): Linear(in_features=12, out_features=48, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
    

In [5]:
num_epochs = 100
learning_rate = 0.0005
weight_decay = 0.0001
momentum = 0.01

In [8]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40])
criterion = nn.CrossEntropyLoss()

In [18]:
checkpoint = torch.load("/kaggle/input/ef-net/ef_net_78.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
loss = checkpoint['loss']
accuracy = checkpoint['accuracy']
average_loss_test = checkpoint['test_accuracy']
average_loss = checkpoint['test_loss']
learning_rate = checkpoint['learning_rate']

0.17334


In [148]:
from torch.utils.data import DataLoader
from torchsummary import summary
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm


for epoch in range(50):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(trainloader, total=len(trainloader), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(trainloader)
    accuracy = correct_predictions / total_samples

    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
      for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss_test += loss.item()

        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)

      accuracy_test = correct_test / total_test
      average_loss_test = total_loss_test / len(testloader)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')
    model_save_path = 'New_efficientnet_cifar10_' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
    scheduler.step()
    if(epoch%10==0):
        torch.save(training_stats, model_save_path)

Epoch [1/100]: 100%|██████████| 782/782 [01:37<00:00,  8.06it/s, accuracy=0.821, loss=0.512]


Epoch [1/100] - Loss: 0.5116, Accuracy: 82.07%, Loss on test data: 0.6649 Accuracy on test data: 77.87 %


Epoch [2/100]: 100%|██████████| 782/782 [01:36<00:00,  8.07it/s, accuracy=0.821, loss=0.506]


Epoch [2/100] - Loss: 0.5065, Accuracy: 82.07%, Loss on test data: 0.6663 Accuracy on test data: 77.33 %


Epoch [3/100]: 100%|██████████| 782/782 [01:38<00:00,  7.97it/s, accuracy=0.821, loss=0.505]


...

Epoch [36/100] - Loss: 0.4603, Accuracy: 83.81%, Loss on test data: 0.6493 Accuracy on test data: 78.23 %


Epoch [37/100]: 100%|██████████| 782/782 [01:35<00:00,  8.15it/s, accuracy=0.837, loss=0.463]


Epoch [37/100] - Loss: 0.4627, Accuracy: 83.69%, Loss on test data: 0.6645 Accuracy on test data: 78.01 %


Epoch [38/100]:  53%|█████▎    | 414/782 [00:51<00:45,  8.10it/s, accuracy=0.839, loss=0.467]


KeyboardInterrupt: 

In [149]:
training_stats = {
    'epoch': epoch+1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'loss': average_loss,
    'accuracy': accuracy,
    'test_accuracy': accuracy_test,
    'test_loss': average_loss_test,
    'learning_rate': optimizer.param_groups[0]["lr"]
}
torch.save(training_stats, "ef_net_78.pth")

In [19]:
print("Accuracy Train: ",accuracy)
print("Accuracy Test: ",accuracy_test)
print("Loss Train: ",average_loss)
print("Loss Test: ",average_loss_test)

Accuracy Train:  0.8369
Accuracy Test:  0.7801
Loss Train:  0.6645258720133714
Loss Test:  0.7801


In [20]:
correct_predictions, total_samples = 0, 0
with torch.no_grad():
  for (inputs, targets) in testloader:
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    __, predicted = outputs.max(1)
    correct_predictions += predicted.eq(targets).sum().item()
    total_samples += targets.size(0)

accuracy = correct_predictions / total_samples
print(f'Accuracy on test data: {100 * correct_predictions / total_samples:.2f} %')

Accuracy on test data: 82.26 %


In [34]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 84.50%
Accuracy on test data car   : 90.90%
Accuracy on test data bird  : 74.20%
Accuracy on test data cat   : 68.50%
Accuracy on test data deer  : 81.10%
Accuracy on test data dog   : 70.90%
Accuracy on test data frog  : 87.40%
Accuracy on test data horse : 85.90%
Accuracy on test data ship  : 89.80%
Accuracy on test data truck : 89.40%
